In [1]:
# 📌 Cell 1: Imports
import pandas as pd
import joblib


In [2]:
# 📌 Cell 2: Load CDR Model
cdr_model = joblib.load(r"D:\Workspace\Projects\CTS\Potential_Customers_for_Upcell\Models\churn_usage_model.pkl")


In [3]:
# 📌 Cell 3: Load New Call Log Excel
cdr_new = pd.read_csv("D:\Workspace\Projects\CTS\Dataset\CDR-Call-Details.csv")  # ensure columns match CDR training features
cdr_features = ["Account Length","VMail Message","Day Mins","Day Calls",
                "Day Charge","Eve Mins","Eve Calls","Eve Charge",
                "Night Mins","Night Calls","Night Charge","Intl Mins","Intl Calls","Intl Charge","CustServ Calls"]


In [4]:
# 📌 Cell 4: Apply CDR Model to New Data
cdr_input = cdr_new[cdr_features].fillna(0).values
cdr_preds = cdr_model.predict(cdr_input)

# Split outputs
cdr_new["Churn_Prediction"] = [p[0] for p in cdr_preds]
cdr_new["Usage_Category"] = [p[1] for p in cdr_preds]


In [ ]:
# 📌 Cell 5: Rule-based Recommendation Function
def recommend_products(row):
    recs = []
    if row["Churn_Prediction"] == 1:
        recs.append("Retention Offer / Free Support Bundle")
    if row["Usage_Category"] == 0:
        recs.append("Basic Retention Offer")
    elif row["Usage_Category"] == 1:
        recs.append("Accessories / Minor Upsell")
    elif row["Usage_Category"] == 2:
        recs.append("Software Products / Premium Add-ons")
    return recs


In [6]:
# 📌 Cell 6: Apply Recommendations
cdr_new["Recommended_Products"] = cdr_new.apply(recommend_products, axis=1)


In [7]:
# 📌 Cell 7: Final Output
final_output = cdr_new[["Phone Number","Churn_Prediction","Usage_Category","Recommended_Products"]]
print(final_output.head())


  Phone Number  Churn_Prediction  Usage_Category         Recommended_Products
0     382-4657                 0               0  [Basic Reten    tion Offer]
1     371-7191                 0               0  [Basic Reten    tion Offer]
2     358-1921                 0               0  [Basic Reten    tion Offer]
3     375-9999                 0               0  [Basic Reten    tion Offer]
4     330-6626                 0               0  [Basic Reten    tion Offer]


In [8]:
# 📌 Cell 8: Optionally Save Output
final_output.to_excel("cdr_recommendations.xlsx", index=False)
